In [8]:
import pandas as pd
import os

ann_data_dir = 'D:\FINKI\8_dps\Project\DATA\physionet_dbs\cudb\\annotations_systemized'
samp_data_dir = 'D:\FINKI\8_dps\Project\DATA\physionet_dbs\cudb\samples'
write_data_dir = 'D:\FINKI\8_dps\Project\DATA\physionet_dbs\cudb\\uniclass_episode_signals_250hz'

if not os.path.exists(write_data_dir):
    os.makedirs(write_data_dir)

In [9]:
for file in os.listdir(ann_data_dir):
    file = str(file)
    print(file)

    ann_file = pd.read_csv(f'{ann_data_dir}\\{file}')
    samp_file = pd.read_csv(f'{samp_data_dir}\\{file}')
    samp_file.columns = ['sample_idx', 'ecg']

    # beat annotations are irrelevant, so they are dismissed
    ann_file = ann_file[~ann_file['episode'].isna()].reset_index(drop = True)


    # the current episode that the segments belong to, and is kept if noise comes as an episode annotation
    ongoing_episode = None

    # iterating episodes
    for ann_idx, ann_row in ann_file.iterrows():

        # if last noise annotation, assume the rest of the signal is fine and noiseless
        if ann_row['episode'] == 'NOISE' and ann_idx == ann_file.shape[0] - 1:
            episode = ongoing_episode
            episode_start_sample = ann_row['sample_idx'] + 1
        # if there are noise anns following the current noise ann, because of ambiguity, dismiss the signal
        elif ann_row['episode'] == 'NOISE' and ann_file.iloc[ann_idx + 1]['episode'] == 'NOISE':
            episode = ongoing_episode
            episode_start_sample = ann_row['sample_idx'] + 1
        elif ann_row['episode'] == 'NOISE' and ann_file.iloc[ann_idx + 1]['episode'] != 'NOISE':
            continue
        # in the normal case when a rhythm episode begins
        else:
            episode = ann_row['episode']
            ongoing_episode = ann_row['episode']
            episode_start_sample = ann_row['sample_idx']

        # if last episode, go to end of file, otherwise end is the start of next episode
        if ann_idx == ann_file.shape[0] - 1:
            episode_end_sample = samp_file.iloc[-1]['sample_idx']
        else:
            episode_end_sample = ann_file.iloc[ann_idx + 1]['sample_idx'] - 1

        clean_episode_signal = samp_file[
            (samp_file['sample_idx'] >= episode_start_sample) &
            (samp_file['sample_idx'] <= episode_end_sample)
        ]

        clean_episode_signal.to_csv(
                f'{write_data_dir}\\'
                f'{file.replace(".csv","")}_'
                f'{episode.replace("(","")}_'
                f'{ann_idx}'
        )

        # break
    #
    # break

cu01.csv
cu02.csv
cu03.csv
cu04.csv
cu05.csv
cu06.csv
cu07.csv
cu08.csv
cu09.csv
cu10.csv
cu11.csv
cu12.csv
cu13.csv
cu14.csv
cu15.csv
cu16.csv
cu17.csv
cu18.csv
cu19.csv
cu20.csv
cu21.csv
cu22.csv
cu23.csv
cu24.csv
cu25.csv
cu26.csv
cu27.csv
cu28.csv
cu29.csv
cu30.csv
cu31.csv
cu32.csv
cu33.csv
cu34.csv
cu35.csv


In [14]:
|